# Example of choropleth for the State of Goias
### Exemplo de choropleth para o Estado de Goiás

In [1]:
import requests
import geopandas as gpd
import pandas as pd
import json
import ipywidgets as widgets
from bqplot import (
    Figure, Map, Mercator, ColorScale, ColorAxis,
    Tooltip
)

In [2]:
#Acesso API Ibge malhas GO
url = 'http://servicodados.ibge.gov.br/api/v2/malhas/52/?resolucao=5'
headers = {'Accept': 'application/vnd.geo+json'}
r = requests.get(url, headers=headers)
#Resposta do API em JSON
j = r.json()
#Visualuar chaves JSON
print(j.keys())
print('############################################')

#Acesso API Ibge Localidades id e nome municipios GO
urlmun = requests.get('http://servicodados.ibge.gov.br/api/v1/localidades/estados/52/municipios')
mun = urlmun.json()
#Visualizar lista mun JSON
#print(mun[1])

dict_keys(['type', 'crs', 'features'])
############################################


In [3]:
#Converter JSON informando keys em GeoDataFrame
goias = gpd.GeoDataFrame.from_features(j['features'])
goias.rename(columns={'codarea': 'id'}, inplace=True)
goias.id = goias.id.astype('int64')
#goias.id.dtype

In [4]:
#DataFrame apenas com id e nome
dfmun = pd.DataFrame(mun, columns=['id', 'nome'])
#dfmun.id.dtype

In [5]:
pibgo = pd.read_excel('data/gopibper.xlsx', usecols=[0,2])
pibgo.rename(columns={'CD_GEOCMU': 'id'}, inplace=True)
#pibgo.id.dtype

In [6]:
#Verificar de Key dos DataFrames são iguais
#Junta do DataFrames 
#Abaixo converte para mesmo formato 
#Merge ou mescla dos DataFrame
gdf = gpd.GeoDataFrame(pd.merge(pd.merge(goias, dfmun), pibgo))

In [7]:
gdf.head(3)

,centroide,id,geometry,nome,pib_percapita
0,"[-52.04564444334576, -18.76783675713795]",5201504,"(POLYGON ((-51.8966 -18.633, -51.8832 -18.6663...",Aporé,57434
1,"[-52.1468663901787, -15.970150495669516]",5201702,"(POLYGON ((-51.8796 -15.8253, -51.8651 -15.827...",Aragarças,10877
2,"[-52.43802923762304, -16.360548844328463]",5203104,"(POLYGON ((-52.2997 -16.167, -52.2913 -16.1899...",Baliza,14688


In [8]:
#Ler JSON
goias = json.loads(gdf.to_json())
#print(type(goias))

In [9]:
gdf['A'] = list(range(len(gdf.index)))

In [ ]:
sc_geo = Mercator(scale_factor=6000, center= (-49.555, -14.8799))
def_tt = Tooltip(fields=['nome', 'pib_percapita'], labels=['Município', 'PIB per capita $'], formats=['', '$,'])

sc_c1 = ColorScale(colors=['Red', 'Gray', 'DeepSkyBlue'])

#sc_c1 = ColorScale(scheme='PuBu')
#sc_c1 = ColorScale(colors=['#63f5cb', 'Gray', 'DeepSkyBlue'])


map_styles = {'color': dict(zip(gdf['A'], gdf['pib_percapita'].astype(float))),
              'scales': {'projection': sc_geo, 'color': sc_c1}, 'colors': {'default_color': 'Grey'}}

axis = ColorAxis(scale=sc_c1, tick_format='.2s', num_ticks=5)

chloro_map = Map(map_data=goias, tooltip=def_tt, **map_styles)
Figure(marks=[chloro_map], axes=[axis],title='Choropleth Example GO', 
       fig_margin={'right':60, 'left':60, 'top':60, 'bottom':60},
      layout=widgets.Layout(width='600px',height='600px'))